In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import time

In [ ]:
## progress bar definition

def print_progressbar(total, i):
    """
    total : total iteration number.
    i : iteration count, starting from 0.
    """
    import math
    step = 25 / total

    # Print the progress bar
    print('\r' + f'Progress: '
        f"[{'=' * int((i+1) * step) + ' ' * (25 - int((i+1) * step))}]"
        f"({math.floor((i+1) * 100 / (total))} %) ({i+1}/{total})",
        end='')
    if (i+1) == total: print("")

## target journal

In [ ]:
# target journal

url = "https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=crohn%27s+disease+early+surgery+prediction&btnG=&oq=crohn"
url = url.replace(f'hl=ko&',f'start=0&')

In [ ]:
# null dataframe build
df = pd.DataFrame(columns=['name', 'author', 'Journal' ,'year', 'info'])


# number of article
num_article = 10

for i in range(0,num_article):
    html = url
    req_info = requests.get(html)
    
    ## page information read using BeautifulSoup
    soup = BeautifulSoup(req_info.content,"html.parser")
    time.sleep(5)
    
    article = soup.find("div").find("div", {'id': 'gs_res_ccl'}).find("div", {'id': 'gs_res_ccl_mid'}).find_all("div", {'class': 'gs_r gs_or gs_scl'})[0]
    
    ## article name
    
    name = article.find("div",{'class':'gs_ri'}).find("h3").text
    
    ## author, Journal, time
    author_time = article.find("div",{'class':'gs_ri'}).find("div",{'class':'gs_a'}).text
    author = author_time.split("\xa0-")[0].split(",")[0:2]
    
    if '\xa0' in author_time:
        Journal = author_time.split("\xa0-")[-1].split(",")[0]
    else:
        Journal = "None"
    
    year = re.findall("\d+",author_time)[0]
    
    ## article info
    if article.find_all("div",{'class':"gs_fl"})[0].text[0]=='[' :
        info = article.find_all("div",{'class':"gs_fl"})[1].text.strip()
    else:
        info = article.find("div",{'class':"gs_fl"}).text.strip()
    
    

    df = df.append({
    'name' : name,
    'author' : author,
    'Journal' : Journal,
    'year' : year,
    'info' : info,
    }, ignore_index=True)
    
    url = url.replace(f'{i}&',f'{i+1}&')
    print_progressbar(num_article, i)
    time.sleep(5)


# dataframe save

df_filename = "crawling_result"

df.to_csv(df_filename, encoding='utf-8-sig', index=False)